# Modern Portfolio Theory: comparison of expected and realized performances

In [1]:
import yfinance as yf

import numpy as np
from numpy.linalg import multi_dot

np.random.seed(42)

import pandas as pd
pd.set_option('display.precision',4)

from datetime import datetime

# Import plotly express for EF plot
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
px.defaults.width, px.defaults.height = 1000,600
pio.templates.default = "plotly_dark"

import cufflinks as cf
cf.set_config_file(offline=True, dimensions=(1000,600),theme='space')

import matplotlib.pyplot as plt

The aim of this project is to analyze historical data from the Italian stock market up to 2020 to determine their expected returns and volatility, then to build possible portfolios at the beginning of yeaar 2021, to compare realized returns up to today with monte carlo simulations based on the return distribution.

## Data analysis
The analysis is focused on 5 of the main italian stocks. The adjusted closing price starting from 2011 is retrieved through Yahoo Finance.

In [2]:
assets = ['STMMI','LDO','TRN','UCG','ENI']
assets.sort()

# Get yahoo tickers
yahooticker = [x+'.MI' for x in assets]

# Fetch data
df = yf.download(yahooticker,start='2011-01-01')['Adj Close']

# Visualize pandas dataframe relative to year 2020
df['2020':'2020']

[*********************100%***********************]  5 of 5 completed


,ENI.MI,LDO.MI,STMMI.MI,TRN.MI,UCG.MI
Date,,,,,
2020-01-02,11.0954,9.9732,24.1750,5.1026,10.8245
2020-01-03,11.1730,10.0732,24.0090,5.0836,10.6961
2020-01-06,11.3393,10.3541,23.4622,5.0371,10.4986
2020-01-07,11.3425,10.4921,24.0578,5.0733,10.5612
2020-01-08,11.2522,10.8063,24.0383,5.1284,10.6961
...,...,...,...,...,...
2020-12-22,7.0888,5.5984,29.7070,5.4865,6.6485
2020-12-23,7.2788,5.7229,29.8546,5.5241,6.7970
2020-12-28,7.2889,5.7638,29.9334,5.6245,6.8032


In [3]:
# Plot price history
df.iplot(kind='line')

Daily returns are computed based on the retrieved data, and then used to evaluate annualized returns and volatility for the whole timespan. The number of trading days is required for scaling daily statistics to their annualized corresponding value. The count here is based on the UK trading calendar ($365 \ \text{days} - 52*2 \ \text{weekend days} - 8\  \text{bank holidays}$). However the trading calendar for the Italian Stock market is different and does not have a fixed number of trading days across different years.

In [4]:
# Number of samples per year
tradingDaysPerYear = 252
# Daily returns
returns = df.pct_change().dropna()
# Annualized statistics
annualReturns = returns.mean()*tradingDaysPerYear
annualVolatility = returns.std()*np.sqrt(tradingDaysPerYear)

# Collecting data in a dataframe
market_statistics = pd.DataFrame({
    'Average annual return': annualReturns,
    'Average annual volatility': annualVolatility
})
market_statistics

,Average annual return,Average annual volatility
ENI.MI,0.0889,0.2737
LDO.MI,0.1259,0.4022
STMMI.MI,0.2467,0.3971
TRN.MI,0.1403,0.2152
UCG.MI,0.0698,0.4620


As the objective of the project is to build a portfolio using historical data and to benchmark its realized returns against their expected performance, the portfolio is built on the first trading day of 2021 using data spanning from 2011 to 2020, assuming no knowledge about future prices. The expected performance of the portfolio is evaluated through Monte Carlo simulation and it is finally compared to the realized performance.

In [5]:
# Compute returns and volatility using data from 2011 to 2020
yearStart = '2011'
yearEnd = '2020'
partial_returns = df[yearStart:yearEnd].pct_change().dropna()
partial_annualReturns = partial_returns.mean()*tradingDaysPerYear
partial_annualVolatility = partial_returns.std()*np.sqrt(tradingDaysPerYear)

partial_statistics = pd.DataFrame({
    'Average annual return': partial_annualReturns,
    'Average annual volatility': partial_annualVolatility
})
partial_statistics

,Average annual return,Average annual volatility
ENI.MI,0.0348,0.2782
LDO.MI,0.0622,0.4203
STMMI.MI,0.2600,0.4089
TRN.MI,0.1418,0.2170
UCG.MI,-0.0519,0.4771


A first forecast of the outcome of this analysis can be drawn by looking at the realized returns and comparing them to the historical data. 

In [8]:
# Compute realized returns between 2021 and 2023
yearStart = '2021'
yearEnd = '2023'
realized_returns = df[yearStart:yearEnd].pct_change().dropna()
realized_annualReturns = realized_returns.mean()*tradingDaysPerYear
realized_annualVolatility = realized_returns.std()*np.sqrt(tradingDaysPerYear)

realized_statistics = pd.DataFrame({
    'Average annual return': realized_annualReturns,
    'Average annual volatility': realized_annualVolatility
})

# Concatenate the two previous dataframes
comparison_df = pd.concat([partial_statistics, realized_statistics] ,axis=1, keys=['Historical data', 'Realized data'])
comparison_df

Historical data                            \
         Average annual return Average annual volatility   
ENI.MI                  0.0348                    0.2782   
LDO.MI                  0.0622                    0.4203   
STMMI.MI                0.2600                    0.4089   
TRN.MI                  0.1418                    0.2170   
UCG.MI                 -0.0519                    0.4771   

                 Realized data                            
         Average annual return Average annual volatility  
ENI.MI                  0.2946                    0.2562  
LDO.MI                  0.3682                    0.3263  
STMMI.MI                0.1837                    0.3497  
TRN.MI                  0.1360                    0.2089  
UCG.MI                  0.5257                    0.4003

In [33]:
# Compare historical and realized data with bar plots
comparison_df.loc[:,(slice(None),'Average annual return')].iplot(kind='bar',title='Historical vs realized returns')
comparison_df.loc[:,(slice(None),'Average annual volatility')].iplot(kind='bar',title='Historical vs realized volatility')

In [43]:
# Compare the normalized stock prices
df['2021':].normalize().iplot()

The Italian stock market deviated from the trends showed in the previous 10 years. ENI, Leonardo and Unicredit Group delivered higher returns than expected, due to specific circumstances such as the energetic crisis, Russian invasion of Ukraine and raising interest rates. Conversely, STMicroelectronics performed worse than expected. However, volatility in these stock prices remained substantially consistent. Therefore the analysis on the portfolio performance will be more meaningful when comparing expected and realized volatility, while realized returns will deviate significantly.

## Portfolio generation

In [40]:
def portfolio_MCgeneration(returns, numPortfolios,short=False):
    # Number of assets in the market
    numAssets = len(returns.columns)

    # Initialize the lists
    rets, vols, wts = [],[],[]

    # Simulate portfolios
    for i in range(numPortfolios):

        # Generate random weights (between -1 and 1 if short selling is allowed, between 0 and 1 otherwise)
        weights = 2*np.random.random(numAssets)-1 if short else np.random.random(numAssets)

        # Normalize to 1 the sum of the weights
        weights /= np.sum(weights)

        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean()*tradingDaysPerYear))
        vols.append(np.sqrt(multi_dot([weights.T,returns.cov()*tradingDaysPerYear,weights])))
        wts.append(weights)
        
    # Create a dataframe for analysis
    data = {'port_rets': rets, 'port_vols': vols}
    for counter, symbol in enumerate(returns.columns.to_list()):
        data[symbol+' weight'] = [w[counter] for w in wts]

    portdf = pd.DataFrame(data)

    # Compute sharpe ratio (assuming zero risk-free rate)
    portdf['sharpe_ratio'] = portdf['port_rets']/portdf['port_vols']

    return round(portdf,4)

In [41]:
# Number of portfolios
numPortfolios = 10000

# Generate portfolios with random weights
portfolioSamples = portfolio_MCgeneration(partial_returns, numPortfolios)
portfolioSamples.head()

,port_rets,port_vols,ENI.MI weight,LDO.MI weight,STMMI.MI weight,TRN.MI weight,UCG.MI weight,sharpe_ratio
0,0.0842,0.2683,0.2506,0.2274,0.1515,0.2129,0.1576,0.3140
1,0.0962,0.2902,0.2102,0.2668,0.2514,0.1088,0.1628,0.3316
2,0.1005,0.3016,0.0062,0.5060,0.1427,0.2565,0.0887,0.3334
3,0.0957,0.2571,0.2958,0.2379,0.1591,0.2335,0.0736,0.3724
4,0.1268,0.2369,0.2989,0.1025,0.2370,0.3466,0.0149,0.5351


In [46]:
# Plot portfolios in mu-sigma plane
fig = px.scatter(
    portfolioSamples, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels={'port_vols': 'Standard deviation', 'port_rets': 'Expected return',
            'sharpe_ratio': 'Sharpe Ratio'},
            title='Monte Carlo generated portfolios'
).update_traces(mode='markers',marker=dict(symbol='cross'))

# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[portfolioSamples.iloc[portfolioSamples.sharpe_ratio.idxmax()]['port_vols']],
    y=[portfolioSamples.iloc[portfolioSamples.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe',
).update(layout_showlegend=False)

fig.add_scatter(
    x=partial_statistics["Average annual volatility"],
    y=partial_statistics["Average annual return"],
    text = [f'Asset: {index}<br>Standard deviation: {vol:.2f}<br>Expected return: {ret:.2f}' 
            for index, vol, ret in zip(partial_statistics.index, partial_statistics["Average annual volatility"], partial_statistics["Average annual return"])],
    hoverinfo='text',
    mode='markers',
    marker=dict(color='green', size=8),
)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [39]:
# Format the plot and save it as html file
fig.update_xaxes(spikecolor="black"), fig.update_yaxes(spikecolor="black")
fig.update_layout(template="plotly_white",)
fig.write_html("markowitz.html")

In [47]:
return_lowerBound, return_upperBound = 0.1, 0.11
portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)]


,port_rets,port_vols,ENI.MI weight,LDO.MI weight,STMMI.MI weight,TRN.MI weight,UCG.MI weight,sharpe_ratio
2,0.1005,0.3016,0.0062,0.5060,0.1427,0.2565,0.0887,0.3334
15,0.1014,0.2765,0.1864,0.1279,0.2629,0.2097,0.2132,0.3668
17,0.1017,0.2581,0.2257,0.1817,0.1977,0.2665,0.1283,0.3939
27,0.1095,0.2684,0.1869,0.1007,0.2733,0.2503,0.1889,0.4079
28,0.1040,0.2984,0.1291,0.1079,0.3284,0.1550,0.2796,0.3486
...,...,...,...,...,...,...,...,...
9956,0.1062,0.3063,0.0400,0.0083,0.3447,0.2385,0.3685,0.3468
9967,0.1060,0.2912,0.2152,0.1141,0.3300,0.1203,0.2204,0.3639
9981,0.1066,0.2678,0.1838,0.2832,0.2057,0.2380,0.0893,0.3980
9994,0.1002,0.2850,0.0760,0.1651,0.2526,0.2474,0.2589,0.3517


In [ ]:
index_maxVol = portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)].port_vols.idxmax()
index_minVol = portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)].port_vols.idxmin()

In [ ]:
portfolio_index = 774
portfolioSamples.iloc[portfolio_index]

retrieve data from source, follow same structure as below

In [ ]:
def simulateStock_MC(S0, mu, sigma, horizon, nTimesteps, nSims):
    # Timestep size
    dt = horizon / nTimesteps

    S = np.zeros((nTimesteps,nSims))
    S[0] = S0

    for i in range(nTimesteps-1):
        w = np.random.standard_normal(nSims)
        S[i+1] = S[i]*(1 + mu*dt + sigma*np.sqrt(dt)*w)

    return S

price_path = pd.DataFrame(simulateStock_MC(31,partial_statistics.loc['STMMI.MI','Average annual return'], partial_statistics.loc['STMMI.MI','Average annual volatility'], 1, 252, 10000))

In [ ]:
price_path.iloc[-1].hist(bins=100)

In [ ]:
import matplotlib.pyplot as plt

ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.plot(range(len(price_path)),price_path.iloc[:,:2000], zs=0, zdir='z', alpha=0.2, color = 'gray')
hist, edges = np.histogram(price_path.iloc[-1,:], bins = 100)
xpos = (edges[:-1] + edges[1:]) / 2
ax.bar(xpos, hist, zs=252, zdir='x', width=np.diff(edges)[0], alpha=0.8)
# plt.plot(price_path.iloc[:,0])
# plt.hist(price_path.iloc[-1], bins = 200,)
# plt.plot(arange(100),arange(100)/2, zs = 152, zdir = 'x')
plt.xlabel('Time steps')
plt.xlim(0,252)
plt.ylabel('Index levels')
plt.title('Monte Carlo Simulated Asset Prices')
plt.grid()
ax.set_xlim(0)
ax.set_zlim(0)
ax.view_init(elev=20,azim=210)

In [ ]:
column_weights = [x+' weight' for x in yahooticker]
weights = portfolioSamples.iloc[portfolio_index][column_weights].to_numpy()

In [ ]:
indices = [index_minVol, index_maxVol,]
weights_set = [portfolioSamples.iloc[index][column_weights].to_numpy() for index in indices]


In [ ]:
def evaluateStock(stock,source,start,end = 'today'):
    end_date = datetime.today().strftime('%Y-%m-%d') if (end == 'today') else end
    return source.loc[start:end_date,stock]

def evaluateMarket(source,start,end = 'today'):
    return pd.DataFrame({stock: evaluateStock(stock, source, start, end) for stock in source.columns})

def evaluatePortfolio(source, weights, initialWealth, start, end = 'today'):
    stockPrices = evaluateMarket(source,start,end)
    nShares = weights*initialWealth/stockPrices.iloc[0].to_numpy()
    portfolio_df = pd.DataFrame({
        'Date': stockPrices.index,
        'portfolioValue': np.dot(stockPrices.to_numpy(), nShares)
    })
    portfolio_df.set_index('Date',inplace=True)
    return portfolio_df
    
def evaluateMultiplePortfolios(source,weights_set, initialWealth, start, end = 'today'):
    dfs = [evaluatePortfolio(source, wts, initialWealth, start, end) for wts in weights_set]
    df = pd.DataFrame({'Portfolio_'+str(i+1): dfs[i]['portfolioValue'] for i in range(len(dfs))})
    return df

In [ ]:

evaluateMultiplePortfolios(df,weights_set, 100, '2021', 'today').iplot(theme='polar')

In [ ]:
evaluatePortfolio(df,weights,100,'2021','2021')

In [ ]:
evaluatePortfolio(df,weights,100,'2021','2021').iplot(asUrl = True, filename = 'html_plot',theme='white')


In [ ]:
portfolioRealizedAnnualReturn = evaluatePortfolio(df,weights,100,'2021').pct_change().dropna().mean()*tradingDaysPerYear
portfolioRealizedAnnualVolatility = evaluatePortfolio(df,weights,100,'2021').pct_change().dropna().std()*np.sqrt(tradingDaysPerYear)

print(f"{portfolioRealizedAnnualReturn = }\n{portfolioRealizedAnnualVolatility =}")

Evaluate single assets performance. Evaluate portfolio performance both in absolute terms and in returns/std. compare with the expected values based on historical data. Write functions to perform such computations for all portfolios. Give conclusion on markowitz theory. Use more data?